In [1]:
import os
import sys
import re
import json
from datasets import (load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict,
    Value,
    Features
)

In [2]:
sys.path.append('../')

In [3]:
import torch
import random
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt

In [4]:
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

2021-12-01 18:46:06.074440: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', use_fast=True)

## Dataset

In [6]:
paper_dataset = load_dataset('metamong1/summarization_paper', 
    use_auth_token='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')

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
law_dataset = load_dataset('metamong1/summarization_law', 
    use_auth_token='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')

Reusing dataset law_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___law_summarization/Paper Summarization/1.2.0/b422baca30e481895dd2b572a7ff9f6c6428725e575fdafb73c0aa1d62356973)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
magazine_dataset = load_dataset('metamong1/summarization_magazine', 
    use_auth_token='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')

Reusing dataset magazine_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___magazine_summarization/Magizine Summarization/1.0.0/506cb41eb0b96b084eafa5dd5fe3b51ff0d1061256700adf1aa92d3b19762c36)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
news_dataset = load_dataset('metamong1/summarization_news', 
    use_auth_token='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')

Reusing dataset news_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___news_summarization/News Summarization/1.0.0/ae25c3215dc878e979d01f1157dbfb014c0a6985fc959ae45eaf10847db75600)


  0%|          | 0/2 [00:00<?, ?it/s]

## Preprocessor

In [10]:
from preprocessor import PaperPreprocessor, DocsPreprocessor, Filter

In [11]:
paper_preprocessor = PaperPreprocessor()
docs_preprocessor = DocsPreprocessor()
data_filter = Filter(title_size=5)

### Paper Dataset

In [12]:
paper_dataset.cleanup_cache_files()
paper_dataset = paper_dataset.map(paper_preprocessor.for_train)
paper_dataset = paper_dataset.filter(data_filter)

  0%|          | 0/73640 [00:00<?, ?ex/s]

  0%|          | 0/18411 [00:00<?, ?ex/s]

  0%|          | 0/74 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

In [13]:
paper_dataset

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 69950
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 17493
    })
})

## Law

In [14]:
law_dataset.cleanup_cache_files()
law_dataset = law_dataset.map(docs_preprocessor.for_train)
law_dataset = law_dataset.filter(data_filter)

  0%|          | 0/23730 [00:00<?, ?ex/s]

  0%|          | 0/5933 [00:00<?, ?ex/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [15]:
law_dataset

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 20921
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 5223
    })
})

### Magazine

In [16]:
magazine_dataset.cleanup_cache_files()
magazine_dataset = magazine_dataset.map(paper_preprocessor.for_train)
magazine_dataset = magazine_dataset.filter(data_filter)

  0%|          | 0/52691 [00:00<?, ?ex/s]

  0%|          | 0/13173 [00:00<?, ?ex/s]

  0%|          | 0/53 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [17]:
magazine_dataset

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 51211
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 12823
    })
})

### News

In [18]:
news_dataset.cleanup_cache_files()
news_dataset = news_dataset.map(docs_preprocessor.for_train)
news_dataset = news_dataset.filter(data_filter)

  0%|          | 0/240628 [00:00<?, ?ex/s]

  0%|          | 0/60157 [00:00<?, ?ex/s]

  0%|          | 0/241 [00:00<?, ?ba/s]

  0%|          | 0/61 [00:00<?, ?ba/s]

In [19]:
news_dataset

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 214237
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 53572
    })
})

## Split Tokens

In [20]:
paper_train = paper_dataset['train']
paper_docs = [doc['text'] for doc in paper_train]

magazine_train = magazine_dataset['train']
magazine_docs = [doc['text'] for doc in magazine_train]

law_train = law_dataset['train']
law_docs = [doc['text'] for doc in law_train]

news_train = news_dataset['train']
news_docs = [doc['text'] for doc in news_train]

In [21]:
total_docs = paper_docs + magazine_docs + law_docs + news_docs

In [22]:
random.shuffle(total_docs)

In [23]:
print('Data Size : %d' %len(total_docs))

Data Size : 356319


In [24]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [26]:
total_sens = []

In [27]:
for doc in tqdm(total_docs) :
    sen_list = sent_tokenize(doc)
    total_sens.extend(sen_list)

  0%|          | 0/356319 [00:00<?, ?it/s]

In [28]:
print('Sentence Size : %d' %len(total_sens))

Sentence Size : 4573278


In [29]:
total_sens[13]

'B 씨의 거주지에서는 GHB 169병(1병 6 ), 조피클론 1008정 등 1억6000만 원 상당의 마약류 및 의약품이 발견돼 경찰이 전량 압수했다.'

In [33]:
sen_tokenized = [tokenizer.tokenize(sen) for sen in tqdm(total_sens)]

  0%|          | 0/4573278 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors


In [34]:
sen_unk = {i:vec for i,vec in enumerate(tqdm(sen_tokenized)) if tokenizer.unk_token in vec}
sen_unk_ids = list(sen_unk.keys())
print('Size : %d' %len(sen_unk))

  0%|          | 0/4573278 [00:00<?, ?it/s]

Size : 66252


## Case1

In [39]:
total_sens[sen_unk_ids[0]]

'실험결과, 모듈러 실험체는 일체형 실험체의 비슷한 휨 성능을 가지고 있었으나, 극한변위는 20% 부족한 모습을 보였다.'

In [37]:
tokenizer.convert_tokens_to_string(sen_unk[sen_unk_ids[0]])

'실험결과, 모듈러 실험체는 일체형 실험체의 비슷한 [UNK] 성능을 가지고 있었으나, 극한변위는 20 % 부족한 모습을 보였다.'

## Case2

In [40]:
total_sens[sen_unk_ids[1]]

'한여름엔 뙤약볕 아래서 아스팔트 열기를 고스란히 느끼며 12시간을 버텨야 했고, 한겨울엔 옷을 5~6겹씩 껴입고도 시린 발을 동동 굴러가며 10시간을 버텼다.'

In [41]:
tokenizer.convert_tokens_to_string(sen_unk[sen_unk_ids[1]])

'한여름엔 [UNK] 아래서 아스팔트 열기를 고스란히 느끼며 12시간을 버텨야 했고, 한겨울엔 옷을 5 ~ 6겹씩 껴입고도 시린 발을 동동 굴러가며 10시간을 [UNK].'

## Case3

In [49]:
total_sens[sen_unk_ids[5]]

'택지개발사업지구 안에 있는 주택 소유자 甲이 사업시행자와 주택에 관한 보상합의를 하면서 인(처, 자녀 및 어머니)과 함께 위 주택에 거주하였다며 사업시행자에게서 인 에 대한 주거이전비를 수령하였는데, 이후 보상대상에서 제외되었던 甲의 아버지 乙이 사업인정고시일 당시 위 주택에서 함께 거주하였다고 주장하면서 사업시행자에게 주거이전비 지급을 청구한 사안에서, 소유자 아닌 원은 사업시행자를 상대로 직접 주거이전비 지급을 구할 수 없다는 이유로, 이와 달리 乙에게 주거이전비 지급청구권이 있다고 본 원심판결에 법리오해의 위법이 있다고 한 사례.'

In [50]:
tokenizer.convert_tokens_to_string(sen_unk[sen_unk_ids[5]])

'택지개발사업지구 안에 있는 주택 소유자 甲 이 사업시행자와 주택에 관한 보상합의를 하면서 인 ( 처, 자녀 및 어머니 ) 과 함께 위 주택에 거주하였다며 사업시행자에게서 인 에 대한 주거이전비를 수령하였는데, 이후 보상대상에서 제외되었던 甲 의 아버지 [UNK] 이 사업인정고시일 당시 위 주택에서 함께 거주하였다고 주장하면서 사업시행자에게 주거이전비 지급을 청구한 사안에서, 소유자 아닌 원은 사업시행자를 상대로 직접 주거이전비 지급을 구할 수 없다는 이유로, 이와 달리 [UNK] 에게 주거이전비 지급청구권이 있다고 본 원심판결에 법리오해의 위법이 있다고 한 사례.'

### Tokenizer Add Tokens

In [94]:
print('Original Sentence')
print(total_sens[sen_unk_ids[12353]])
print('')
print('Tokenized Sentence')
print(tokenizer.convert_tokens_to_string(sen_unk[sen_unk_ids[12353]]))

Original Sentence
반면 아크릴 수지계 에멀젼 등을 습식코팅한 방수코팅 종이의 경우는 재활용성은 높지만, 폴리에틸렌 방수코팅 종이에 비해 방수성이나 열접착성, 용기성형성이 미흡했다.

Tokenized Sentence
반면 아크릴 수지계 [UNK] 등을 습식코팅한 방수코팅 종이의 경우는 재활용성은 높지만, 폴리에틸렌 방수코팅 종이에 비해 방수성이나 열접착성, 용기성형성이 미흡했다.


In [95]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', use_fast=True)
tokenizer.add_tokens(['젼'])
tokenizer.tokenize('에멀젼')

['에', '##멀', '젼']

In [96]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', use_fast=True)
tokenizer.add_tokens(['##젼'])
tokenizer.tokenize('에멀젼')

['[UNK]']

In [93]:
print('Original Sentence')
print(total_sens[sen_unk_ids[21]])
print('')
print('Tokenized Sentence')
print(tokenizer.convert_tokens_to_string(sen_unk[sen_unk_ids[21]]))

Original Sentence
도교육청은 지난해부터 '등굣길 1 걷기 캠페인'을 추진하고 있는데 안전한 통학로 확보가 우선돼야 함은 두말할 필요가 없다.

Tokenized Sentence
도교육청은 지난해부터'[UNK] 1 걷기 캠페인'을 추진하고 있는데 안전한 통학로 확보가 우선돼야 함은 두말할 필요가 없다.


In [97]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', use_fast=True)
tokenizer.add_tokens(['굣'])
tokenizer.tokenize('등굣길')

['등', '굣', '길']

In [98]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', use_fast=True)
tokenizer.add_tokens(['##굣'])
tokenizer.tokenize('등굣길')

['[UNK]']

In [111]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', use_fast=True)
vocab_set = tokenizer.vocab

In [103]:
start_id = ord('가')
end_id = ord('힣')

In [120]:
size = 0

for i in range(start_id, end_id+1) :
    ch1 = chr(i)
    ch2 = '##' + ch1
    flag = ch1 in vocab_set and ch2 not in vocab_set

    if flag == True :
        size += 1 

print('원래 글자는 있고 ##붙인 글자는 없는 경우') # 굣 - O,  ##굣 - X
print('Size : %d' %size)

원래 글자는 있고 ##붙인 글자는 없는 경우
Size : 0


In [121]:
size = 0
for i in range(start_id, end_id+1) :
    ch1 = chr(i)
    ch2 = '##' + ch1
    flag = ch1 not in vocab_set and ch2 in vocab_set
    
    if flag == True :
        size += 1

print('원래 글자는 없고 ##붙인 글자는 있는 경우') # 굣 - X,  ##굣 - O
print('Size : %d' %size)

원래 글자는 없고 ##붙인 글자는 있는 경우
Size : 0


In [122]:
size = 0 

for i in range(start_id, end_id+1) :
    ch1 = chr(i)
    ch2 = '##' + ch1
    flag = ch1 in vocab_set and ch2 in vocab_set
    
    if flag == True :
        size += 1

print('원래 글자는 있고 ##붙인 글자는 있는 경우') # 굣 - O,  ##굣 - O
print('Size : %d' %size)

원래 글자는 있고 ##붙인 글자는 있는 경우
Size : 1431


## Select Characters

In [123]:
from konlpy.tag import Mecab
mecab = Mecab()

In [125]:
unk_words = []

In [126]:
for idx in tqdm(sen_unk_ids) :
    sen = total_sens[idx]
    tok_list = mecab.morphs(sen)

    for tok in tok_list :
        if tokenizer.unk_token_id in tokenizer.encode(tok) :
            unk_words.append(tok)

  0%|          | 0/66252 [00:00<?, ?it/s]

In [127]:
unk_words[:10]

['휨', '뙤약볕', '버텼', '듁', '짰', '묙', '꿩', '乙', '乙', '헝겊']

In [128]:
unk_chars = []

In [129]:
for word in tqdm(unk_words) :
    for ch in word :
        if tokenizer.convert_tokens_to_ids(ch) == tokenizer.unk_token_id :
            unk_chars.append(ch)

  0%|          | 0/108285 [00:00<?, ?it/s]

In [130]:
unk_chars[:10]

['휨', '뙤', '텼', '듁', '짰', '묙', '꿩', '乙', '乙', '겊']

In [131]:
unk_ch_counter = collections.Counter()
unk_ch_counter.update(unk_chars)

In [133]:
unk_char_items = sorted(unk_ch_counter.items(), key=lambda x : x[1], reverse=True)

In [136]:
unk_char_items[:10]

[('乙', 4896),
 ('丙', 1959),
 ('챗', 925),
 ('펭', 813),
 ('엊', 770),
 ('丁', 709),
 ('탰', 647),
 ('휨', 643),
 ('켐', 632),
 ('갭', 514)]

In [137]:
unk_ch_list = [item[0] for item in unk_char_items]
unk_ch_count = [item[1] for item in unk_char_items]

In [138]:
extra_unk_chars = pd.DataFrame({'Character' : unk_ch_list, 'Count' : unk_ch_count})

In [140]:
def check_korean(ch) :
    if ord(ch) in range(ord('가'), ord('힣')+1) :
        return True
    else :
        return False 

In [144]:
extra_unk_chars['KoreanFlag'] = extra_unk_chars['Character'].map(check_korean)

In [145]:
extra_unk_chars.head()

,Character,Count,KoreanFlag
0,乙,4896,False
1,丙,1959,False
2,챗,925,True
3,펭,813,True
4,엊,770,True


In [146]:
extra_unk_chars.to_csv('../Tokenizer/extra_characters.csv')